In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier

#1

print('Question 1:')
df=pd.read_csv('seismic_bumps.csv')
#dropping 8 columns as most values are 0 and are giving singular cov matrix
df1=df.drop(df.loc[:,'nbumps':'nbumps89'].columns,axis = 1) 
df10=df1[df1['class']==0]
df11=df1[df1['class']==1]
x_label0=df10['class']
x_label1=df11['class']
y0 = x_label0  #for convenience 
y1 = x_label1  #for convenience 
x0=df10.drop(['class'],axis = 1)
x1=df11.drop(['class'],axis = 1)
x_train0,x_test0,y_train0,y_test0=train_test_split(x0,y0,test_size=0.3,random_state=42, shuffle=True)
x_train1,x_test1,y_train1,y_test1=train_test_split(x1,y1,test_size=0.3,random_state=42, shuffle=True)
x_train = pd.concat((x_train0, x_train1), axis = 0)
x_test = pd.concat((x_test0, x_test1), axis = 0)
y_train = pd.concat((y_train0, y_train1), axis = 0)
y_test = pd.concat((y_test0, y_test1), axis = 0)
train = pd.concat((x_train, y_train), axis = 1)
test = pd.concat((x_test, y_test), axis = 1)
train.to_csv('seismic-bumps-train.csv')
test.to_csv('seismic-bumps-test.csv')

# a & b

print('a & b:')
def knnc( x_train, y_train, x_test, y_test): #defining knnc function to see accuracies of different k values
    for k in range(1,6,2):
        knn1 = KNeighborsClassifier(n_neighbors=k)
        knn1.fit(x_train, y_train)
        y_pred = knn1.predict(x_test)
        print('Confusion matrix for k =', k)
        print(confusion_matrix(y_test, y_pred))
        a = accuracy_score(y_test, y_pred)
        print("Accuracy Score is",a,'\n')

knnc( x_train, y_train, x_test, y_test) #function call
print('Note:maximum accuracy for k=5\n')
#2

print('Question 2:')
#Min_Max Normalization exluding class attribute
x_trainnormal=((x_train-x_train.min())/(x_train.max()-x_train.min()))*(1-0)+0 
train_normal = pd.concat((x_trainnormal,y_train),axis = 1) #included class attribute
#Min_Max Normalization of test data using min, max of train dataset
x_testnormal=((x_test-x_train.min())/(x_train.max()-x_train.min()))*(1-0)+0
test_normal = pd.concat((x_testnormal,y_test),axis = 1)
train_normal.to_csv('seismic-bumps-train-Normalised.csv')
test_normal.to_csv(' seismic-bumps-test-normalised.csv')

# a & b

knnc(x_trainnormal, y_train,x_testnormal,y_test)
print('Note:maximum accuracy for k=5\n')

#3
print('Question 3:')

#4
print('Question 4:')



Question 1:
a & b:
Confusion matrix for k = 1
[[671  54]
 [ 46   5]]
Accuracy Score is 0.8711340206185567 

Confusion matrix for k = 3
[[707  18]
 [ 47   4]]
Accuracy Score is 0.9162371134020618 

Confusion matrix for k = 5
[[718   7]
 [ 46   5]]
Accuracy Score is 0.9317010309278351 

Note:maximum accuracy for k=5

Question 2:
Confusion matrix for k = 1
[[678  47]
 [ 42   9]]
Accuracy Score is 0.8853092783505154 

Confusion matrix for k = 3
[[705  20]
 [ 44   7]]
Accuracy Score is 0.9175257731958762 

Confusion matrix for k = 5
[[718   7]
 [ 48   3]]
Accuracy Score is 0.9291237113402062 

Note:maximum accuracy for k=5

Question 3:
Question 4:
